In [1]:
import pickle
import os

In [2]:
max_new_words = 300
general_constraint = f"Always provide answers that are short, concise and to the point. Keep your answer within {max_new_words} words. Ensure clarity and consistency in your responses, avoiding contradictory statements or redundancy."

In [3]:
def save_prompt(prompt, filename, path):
    with open(path + filename, 'wb') as f:
        pickle.dump(prompt, f)
        
path = '../data/prompts/'

In [4]:
if not os.path.exists(path):
    os.makedirs(path)

In [5]:
sdg_description = open(f"{path}SDG_Description.txt", "r").read()

replacements_default = {
    'system': {
        'sdg_description': sdg_description,
        'general_constraint': general_constraint},
    'user': {
        'general_constraint': general_constraint}
}

generation_args_default = {
    'max_new_tokens': max_new_words*1.25,
    'do_sample': True,
    'temperature': 0.8,
    'top_k': 30,
    'top_p': 0.95,
    'num_return_sequences': 1,
    'sep_token': 'token_not_set'
}

# pickle
save_prompt(replacements_default, 'replacements_default.pickle', path)
    
save_prompt(generation_args_default, 'generation_args_default.pickle', path)

In [6]:
core_themes_message = [
    {
        "role": "system",
        "content": "You are an AI expert in scientific abstract analysis. {general_constraint}"
    },
    {
        "role": "user",
        "content": "Summarize the primary topics, methods, and conclusions of the provided scientific abstract. Abstract:\n\"{Abstract_Text}\""
    }
]

save_prompt(core_themes_message, 'core_themes_extraction.pickle', path)

In [7]:
sdg_concepts_message_core_themes = [
    {
        "role": "system",
        "content": "You are an AI expert trained across multiple disciplines relevant to the Sustainable Development Goals (SDGs), including environmental, social, economic, cultural, and technological fields. {general_constraint}\n {sdg_description}"
    },
    {
        "role": "user",
        "content": "Given the identified core themes:\n\"{Core_Themes}\"\n\nAnalyze the following abstract for its direct relevance to the SDGs. Scientific Abstract and Title:\n\"{Abstract_Text}\""
    }
]

save_prompt(sdg_concepts_message_core_themes, 'sdg_concepts_extraction_core_themes.pickle', path)

In [8]:
retrieval_message_core_themes = [
    {
        "role": "system",
        "content": "You are an AI trained across multiple disciplines relevant to the Sustainable Development Goals (SDGs), including environmental, social, economic, cultural, technological fields, and scientific analysis. Critically and objectively analyze texts for their contribution to Sustainable Development Goals (SDGs). Summarize the main goal of the provided SDG and assess whether the abstract directly contributes towards achieving this SDG. Justify instances where the abstract does not directly contribute to the SDG. {general_constraint}"
    },
    {
        "role": "user",
        "content": "Given the identified core themes:\n\"{Core_Themes}\"\n\nAnalyze the following scientific abstract for its direct relevance to the SDGs \"{closest_abstract_sdgs}, {closest_sdg_centroid}\". Scientific Abstract and Title:\n\"{Abstract_Text}\"\n\n"
    }
]

save_prompt(retrieval_message_core_themes, 'retrieval_relevance_extraction_core_themes.pickle', path)

In [9]:
merge_message_core_themes_similarity = [
    {
        "role": "system",
        "content": "You are an AI specialized in synthesizing and summarizing complex texts related to the same scientific abstract. Your task is to merge various inputs into a single, coherent summary. Focus specifically on the directly impacted Sustainable Development Goals (SDGs). Exclude irrelevant or indirectly affected SDGs, and correct any inconsistencies or inaccuracies in the information. All inputs provided are aspects of the same scientific abstract. {general_constraint}"
    },
    {
        "role": "user",
        "content": "Here are the texts to be summarized, all pertaining to the same scientific abstract: \n\n1. Scientific Abstract and Title:\n\"{Abstract_Text}\"\n\n2. Core Themes of the Scientific Abstract:\n\"{Core_Themes}\"\n\n3. SDG Concepts of the Scientific Abstract:\n\"{SDG_Concepts}\"\n\n4. Additional Information about the Scientific Abstract:\n\"{Retrieval}\"\n\nCreate a unified summary of these inputs, emphasizing the directly impacted SDGs and rectifying any inaccuracies or inconsistencies."
    }
]

save_prompt(merge_message_core_themes_similarity, 'second_merge_prompt_similarity.pickle', path)

In [10]:
defect_analysis_message = [
    {
        "role": "system",
        "content": "You are an AI trained in critical analysis and sustainable development goals (SDGs). Your task is to critically analyze the provided abstract against the provisional classification for alignment with relevant SDGs. Highlight any discrepancies, weak, or indirect connections. {general_constraint} Refer to the provided SDG descriptions for accurate comparisons.\n\n{sdg_description}\n\n{general_constraint}"
    },
    {
        "role": "user",
        "content": "Scientific Abstract and Title:\n\"{Abstract_Text}\"\n\nProvisional Classification:\n\"{Response_Text}\"\n\nAnalyze the provisional classification and identify any discrepancies, weak, or indirect connections. Justify your choices and provide enhancements."
    }
]

save_prompt(defect_analysis_message, 'defect_analysis_prompt.pickle', path)

In [11]:
guided_optimization_message = [
    {
        "role": "system",
        "content": "You are an AI tasked with optimizing classifications related to Sustainable Development Goals (SDGs). Revise the provided provisional classification by addressing the identified enhancements. Ensure that your revised classification only includes directly relevant SDGs and clearly justifies the direct relevance of each selected SDG. Adjust the rankings and explanations to reflect a more accurate alignment with the SDGs.\n\n{general_constraint}"
    },
    {
        "role": "user",
        "content": "Scientific Abstract and Title:\n\"{Abstract_Text}\"\n\nProvisional Classification:\n\"{Defective_Response_Text}\"\n\nIdentified Enhancements:\n\"{Identified_Issues_Text}\"\n\nRevise the provisional classification to include only directly relevant SDGs, justifying each choice and create a ranking to accurately resemble the SDG alignment."
    }
]

save_prompt(guided_optimization_message, 'guided_optimization_prompt.pickle', path)

In [12]:
result_message_single_sdg_finetuned = [
    {
        "role": "system",
        "content": "As a precise AI, your specific function is to identify the single, most pertinent Sustainable Development Goal (SDG) from those mentioned in the input in relation to the abstract. Select the one SDG that is referenced in the input text as the primary SDG the research contributes to based on the content of the abstract. Your response must be formatted as 'SDG X', where 'X' is the number of the most relevant SDG from those mentioned in the input. Ensure your response strictly adheres to this format and excludes any justification or additional information.\n\n"
    },
    {
        "role": "user",
        "content": "Input:\n[input]\"{DSRE_Response}\"[/input]\n"
    }
]

save_prompt(result_message_single_sdg_finetuned, 'single_sdg_finetuned.pickle', path)

In [13]:
result_message_all_sdg_finetuned = [
    {
        "role": "system",
        "content": "Analyze the provided input to identify which Sustainable Development Goals (SDGs) are directly impacted by the scientific abstract. Extract only the SDGs with clear contributions as mentioned in the input. Present your findings as a list of SDG numbers, separated by commas. Your response should be only this list, without additional justification or explanation."
    },
    {
        "role": "user",
        "content": "Input:\n[input]\"{DSRE_Response}\"[/input]\n"
    }
]

save_prompt(result_message_all_sdg_finetuned, 'all_sdg_finetuned.pickle', path)